# 7 - Multi-Agent
## Agent calling another Agent - Agent as a Tool
- This code snippet demonstrates the "Agent as a Tool" pattern in Google ADK, allowing one agent to invoke another agent's capabilities like a function call. 
- It defines a target agent, ImageGeneratorAgent, which simulates generating an image. This ImageGeneratorAgent instance is then wrapped within an AgentTool. 
- A parent agent, Artist, is configured as an LlmAgent and given access to the image_tool in its tools list. When the Artist LLM decides to use the image generation capability, it generates a function call targeting the ImageGen tool. The ADK framework intercepts this call, executes the wrapped ImageGeneratorAgent via the AgentTool, and returns the agent's final output (the simulated image part) back to the Artist agent as the tool result. 
- This pattern provides a structured way for LLM agents to leverage the specialized functionalities of other agents.

In [1]:
# Conceptual Setup: Agent as a Tool
from google.adk.agents import LlmAgent, BaseAgent
from google.adk.tools import agent_tool
from pydantic import BaseModel

# Define a target agent (could be LlmAgent or custom BaseAgent)
class ImageGeneratorAgent(BaseAgent): # Example custom agent
    name: str = "ImageGen"
    description: str = "Generates an image based on a prompt."
    # ... internal logic ...
    async def _run_async_impl(self, ctx): # Simplified run logic
        prompt = ctx.session.state.get("image_prompt", "default prompt")
        # ... generate image bytes ...
        image_bytes = b"..."
        yield Event(author=self.name, content=types.Content(parts=[types.Part.from_bytes(image_bytes, "image/png")]))

image_agent = ImageGeneratorAgent()
image_tool = agent_tool.AgentTool(agent=image_agent) # Wrap the agent

# Parent agent uses the AgentTool
artist_agent = LlmAgent(
    name="Artist",
    model="gemini-2.0-flash",
    instruction="Create a prompt and use the ImageGen tool to generate the image.",
    tools=[image_tool] # Include the AgentTool
)
# Artist LLM generates a prompt, then calls:
# FunctionCall(name='ImageGen', args={'image_prompt': 'a cat wearing a hat'})
# Framework calls image_tool.run_async(...), which runs ImageGeneratorAgent.
# The resulting image Part is returned to the Artist agent as the tool result.